# 100k events ; using CKKW

In [19]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0
    initial_events_sq = events_passed_met_cut_sq = events_passed_pt_eta_cut_sq = events_passed_jets_criteria_sq = events_passed_delta_phi_cut_sq = events_passed_leading_jet_cut_sq = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = {200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    # Initialize dictionary to store squared sum of event weights for each MET cut value
    met_cuts_sq = {met_cut: 0 for met_cut in met_cuts}

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt = leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
        initial_events_sq += (branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights) ** 2

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            events_passed_met_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_pt_eta_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_jets_criteria_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                    events_passed_delta_phi_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                                # Update squared sum of event weights for each MET cut value
                                met_cuts_sq[met_cut] += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2
                                events_passed_leading_jet_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")

    # Calculate statistical uncertainties for each cut
    initial_events_uncer = np.sqrt(initial_events_sq)
    events_passed_met_cut_uncer = np.sqrt(events_passed_met_cut_sq)
    events_passed_pt_eta_cut_uncer = np.sqrt(events_passed_pt_eta_cut_sq)
    events_passed_jets_criteria_uncer = np.sqrt(events_passed_jets_criteria_sq)
    events_passed_delta_phi_cut_uncer = np.sqrt(events_passed_delta_phi_cut_sq)
    events_passed_leading_jet_cut_uncer = np.sqrt(events_passed_leading_jet_cut_sq)

    print(f"\nInitial number of events: {initial_events:.2f} +/- {initial_events_uncer:.2f} ")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {events_passed_met_cut_uncer:.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {events_passed_pt_eta_cut_uncer:.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {events_passed_jets_criteria_uncer:.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {events_passed_delta_phi_cut_uncer:.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {events_passed_leading_jet_cut_uncer:.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    ratios_uncer = {} # Dictionary to store the ratios uncertainty
    
    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            ratio_uncer = np.sqrt(met_cuts_sq[met_cut]) / paper_met_cuts[met_cut]
            ratios[met_cut] = ratio
            ratios_uncer[met_cut] = ratio_uncer
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f} +/- {np.sqrt(met_cuts_sq[met_cut]):.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncer:.2f}")
            
            # Calculate uncer for the ratio
            # ratio_uncer = ratio * np.sqrt((np.sqrt(met_cuts[met_cut]) / met_cuts[met_cut]) ** 2 + (np.sqrt(paper_met_cuts[met_cut]) / paper_met_cuts[met_cut]) ** 2)
            # ratios[met_cut] = (ratio, ratio_uncer)

    # Find the MET cut with the maximum ratio
    max_ratio_met_cut = max(ratios, key=ratios.get)
    max_ratio_uncer_met_cut = max(ratios_uncer, key=ratios_uncer.get)
    # max_ratio_met_cut = max(ratios, key=lambda x: ratios[x][0])

    # Calculate gluino mass point
    gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
    gluino_mass_point_uncer = cross_section * (paper_met_cuts[max_ratio_uncer_met_cut] / met_cuts[max_ratio_uncer_met_cut])

    print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point, "+/-", gluino_mass_point_uncer)



In [18]:
analyze_specific_points("root_files/N590_G600/tag_1_delphes_events.root", 0.113E+02)


Analysis results for root file: root_files/N590_G600/tag_1_delphes_events.root

Initial number of events: 1582000.00 +/- 5725.23 

Number of events passed MET > 200 GeV cut: 397265.10 +/- 3007.63

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 396590.97 +/- 3005.15

Number of events passed number of jets <= 4 cut: 336623.74 +/- 2761.82

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 269362.59 +/- 2465.46

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 233841.09 +/- 4494.36

For SR MET > 200 GeV: S_Exp: 233841.09 +/- 2310.59, S_Obs: 102274, and ratio: 2.29 +/- 0.02

For SR MET > 250 GeV: S_Exp: 185171.01 +/- 2076.52, S_Obs: 41158, and ratio: 4.50 +/- 0.05

For SR MET > 300 GeV: S_Exp: 140390.80 +/- 1815.40, S_Obs: 20893, and ratio: 6.72 +/- 0.09

For SR MET > 350 GeV: S_Exp: 106721.88 +/- 1582.92, S_Obs: 11937, and ratio: 8.94 +/- 0.13

For SR MET > 400 GeV: S_Exp: 81767.95 +/- 1384.13, S_Obs: 7214, and ratio: 11.33 +/- 0.

In [1]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")
    print(f"\nInitial number of events: {initial_events:.2f}")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {np.sqrt(events_passed_met_cut):.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {np.sqrt(events_passed_pt_eta_cut):.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {np.sqrt(events_passed_jets_criteria):.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {np.sqrt(events_passed_delta_phi_cut):.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {np.sqrt(events_passed_leading_jet_cut):.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    
    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            # ratio_uncer =         sqrt(  numerator_uncer / numerator)**2 + (denominator_uncer / denominator)**2)
            # ratio_uncer = ratio * sqrt(    (sqrt(numerator)    / numerator)**2 + (    sqrt(denominator)   / denominator)**2)
            # Numerator corresponds to met_cuts[met_cut]
            # Denominator corresponds to paper_met_cuts[met_cut]
            ratio_uncer = ratio * np.sqrt((np.sqrt(met_cuts[met_cut]) / met_cuts[met_cut]) ** 2 + (np.sqrt(paper_met_cuts[met_cut]) / paper_met_cuts[met_cut]) ** 2)
            ratios[met_cut] = (ratio, ratio_uncer)
            #ratios[met_cut] = ratio
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f} +/- {np.sqrt(met_cuts[met_cut]):.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncer:.2f}")

    # max_ratio_met_cut = max(ratios, key=ratios.get)
    max_ratio_met_cut = max(ratios, key=lambda x: ratios[x][0])

    gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])

    print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Welcome to JupyROOT 6.26/06


# For N1 590 GeV ; gl 600 GeV

In [2]:
analyze_specific_points("root_files/N590_G600/tag_1_delphes_events.root", 0.113E+02)


Analysis results for root file: root_files/N590_G600/tag_1_delphes_events.root

Initial number of events: 1582000.00

Number of events passed MET > 200 GeV cut: 397265.10 +/- 630.29

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 396590.97 +/- 629.75

Number of events passed number of jets <= 4 cut: 336623.74 +/- 580.19

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 269362.59 +/- 519.00

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 233841.09 +/- 483.57

For SR MET > 200 GeV: S_Exp: 233841.09 +/- 483.57, S_Obs: 102274, and ratio: 2.29 +/- 0.01

For SR MET > 250 GeV: S_Exp: 185171.01 +/- 430.32, S_Obs: 41158, and ratio: 4.50 +/- 0.02

For SR MET > 300 GeV: S_Exp: 140390.80 +/- 374.69, S_Obs: 20893, and ratio: 6.72 +/- 0.05

For SR MET > 350 GeV: S_Exp: 106721.88 +/- 326.68, S_Obs: 11937, and ratio: 8.94 +/- 0.09

For SR MET > 400 GeV: S_Exp: 81767.95 +/- 285.95, S_Obs: 7214, and ratio: 11.33 +/- 0.14

For SR MET > 500 Ge

# For N1 965 GeV ; gl 975 GeV

In [3]:
analyze_specific_points("root_files/N965_G975/tag_1_delphes_events.root", 0.462)


Analysis results for root file: root_files/N965_G975/tag_1_delphes_events.root

Initial number of events: 64680.00

Number of events passed MET > 200 GeV cut: 16261.99 +/- 127.52

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 16227.51 +/- 127.39

Number of events passed number of jets <= 4 cut: 13812.97 +/- 117.53

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 11075.79 +/- 105.24

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 9629.54 +/- 98.13

For SR MET > 200 GeV: S_Exp: 9629.54 +/- 98.13, S_Obs: 102274, and ratio: 0.09 +/- 0.00

For SR MET > 250 GeV: S_Exp: 7594.71 +/- 87.15, S_Obs: 41158, and ratio: 0.18 +/- 0.00

For SR MET > 300 GeV: S_Exp: 5801.40 +/- 76.17, S_Obs: 20893, and ratio: 0.28 +/- 0.00

For SR MET > 350 GeV: S_Exp: 4395.24 +/- 66.30, S_Obs: 11937, and ratio: 0.37 +/- 0.01

For SR MET > 400 GeV: S_Exp: 3364.46 +/- 58.00, S_Obs: 7214, and ratio: 0.47 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1971.09 +/- 44

# For N1 850 GeV ; gl 1000 GeV

In [4]:
analyze_specific_points("root_files/N850_G1000/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N850_G1000/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13528.20 +/- 116.31

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13499.86 +/- 116.19

Number of events passed number of jets <= 4 cut: 11459.50 +/- 107.05

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9174.01 +/- 95.78

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7964.87 +/- 89.25

For SR MET > 200 GeV: S_Exp: 7964.87 +/- 89.25, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6302.44 +/- 79.39, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4774.13 +/- 69.10, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3622.55 +/- 60.19, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2777.26 +/- 52.70, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1645.79 +/- 40.

# For N1 900 GeV ; gl 1000 GeV

In [5]:
analyze_specific_points("root_files/N900_G1000/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N900_G1000/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13585.72 +/- 116.56

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13556.38 +/- 116.43

Number of events passed number of jets <= 4 cut: 11520.81 +/- 107.34

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9255.22 +/- 96.20

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8020.01 +/- 89.55

For SR MET > 200 GeV: S_Exp: 8020.01 +/- 89.55, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6310.31 +/- 79.44, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4815.42 +/- 69.39, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3650.73 +/- 60.42, S_Obs: 11937, and ratio: 0.31 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2809.05 +/- 53.00, S_Obs: 7214, and ratio: 0.39 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1668.44 +/- 40.

# For N1 950 GeV ; gl 1000 GeV

In [6]:
analyze_specific_points("root_files/N950_G1000/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N950_G1000/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13547.85 +/- 116.40

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13519.67 +/- 116.27

Number of events passed number of jets <= 4 cut: 11470.67 +/- 107.10

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9213.26 +/- 95.99

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8004.70 +/- 89.47

For SR MET > 200 GeV: S_Exp: 8004.70 +/- 89.47, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6315.89 +/- 79.47, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4811.75 +/- 69.37, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3634.22 +/- 60.28, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2769.77 +/- 52.63, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1631.25 +/- 40.

# For N1 990 GeV ; gl 1000 GeV

In [7]:
analyze_specific_points("root_files/N990_G1000/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N990_G1000/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13549.63 +/- 116.40

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13524.41 +/- 116.29

Number of events passed number of jets <= 4 cut: 11443.87 +/- 106.98

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9223.06 +/- 96.04

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7988.20 +/- 89.38

For SR MET > 200 GeV: S_Exp: 7988.20 +/- 89.38, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6318.24 +/- 79.49, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4827.87 +/- 69.48, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3643.84 +/- 60.36, S_Obs: 11937, and ratio: 0.31 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2776.32 +/- 52.69, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1652.42 +/- 40.

# For N1 995 GeV ; gl 1000 GeV

In [8]:
analyze_specific_points("root_files/N995_G1000/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N995_G1000/tag_1_delphes_events.root

Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13528.92 +/- 116.31

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13504.86 +/- 116.21

Number of events passed number of jets <= 4 cut: 11415.72 +/- 106.84

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9186.02 +/- 95.84

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7994.23 +/- 89.41

For SR MET > 200 GeV: S_Exp: 7994.23 +/- 89.41, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6300.65 +/- 79.38, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4796.62 +/- 69.26, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3618.33 +/- 60.15, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2763.36 +/- 52.57, S_Obs: 7214, and ratio: 0.38 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1656.42 +/- 40.

# For N1 1015 GeV ; gl 1025 GeV

In [9]:
analyze_specific_points("root_files/N1015_G1025/tag_1_delphes_events.root", 0.322)


Analysis results for root file: root_files/N1015_G1025/tag_1_delphes_events.root

Initial number of events: 45080.00

Number of events passed MET > 200 GeV cut: 11300.30 +/- 106.30

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 11277.46 +/- 106.20

Number of events passed number of jets <= 4 cut: 9564.76 +/- 97.80

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7647.05 +/- 87.45

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 6623.78 +/- 81.39

For SR MET > 200 GeV: S_Exp: 6623.78 +/- 81.39, S_Obs: 102274, and ratio: 0.06 +/- 0.00

For SR MET > 250 GeV: S_Exp: 5181.02 +/- 71.98, S_Obs: 41158, and ratio: 0.13 +/- 0.00

For SR MET > 300 GeV: S_Exp: 3974.80 +/- 63.05, S_Obs: 20893, and ratio: 0.19 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3021.88 +/- 54.97, S_Obs: 11937, and ratio: 0.25 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2303.71 +/- 48.00, S_Obs: 7214, and ratio: 0.32 +/- 0.01

For SR MET > 500 GeV: S_Exp: 1371.91 +/- 37.0